In [2]:
#IMPORTING REQUIRED MODULES

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from datetime import datetime

#LOADING THE COVID CSV FILE AND PARSING THE DATE COLUMN AS DATE DATATYPE

newdf=pd.read_csv(r'districts.csv',parse_dates = ['Date'])

In [3]:
#REMOVED THE ROWS THAT HAVE DISTRICTNAME THAT ARE NOT DISTRICT NAME LIKE Unknown,Other State,Other Region....


df=newdf[newdf['District']!='Unknown']
df=df[df['District']!='Other State']
df=df[df['District']!='Other Region']
df=df[df['District']!='Italians']
df=df[df['District']!='Evacuees']
df=df[df['District']!='BSF Camp']
df=df[df['District']!='Airport Quarantine']
df=df[df['District']!='Railway Quarantine']
df=df[df['District']!='Foreign Evacuees']
df=df[df['District']!='Capital Complex']
df=df[df['District']!='Others']
df=df[df['District']!='State Pool']

#REMOVING UNNECESSARY COLUMNS 

del df['Other']
del df['Tested']

#TAKING ONLY ROWS THAT HAVE DATE BEFORE AUG 14 2021

df=df[df['Date']<'2021-08-15']

In [4]:
#STORING THE STATE NAMES INTO A SEPERATE LIST

states=list(df['State'])

#CREATING A LIST CONTAINING ELEMETS THAT ARE STATE NAME AND STATEID

change=[['Andaman and Nicobar Islands', 'AN'],[ 'Andhra Pradesh',  'AP'],[ 'Arunachal Pradesh', 'AR'],[ 'Assam',  'AS'],[ 'Bihar',  'BR'],
 [ 'Chandigarh' ,'CH'],[ 'Chhattisgarh' , 'CT'], ['Dadra and Nagar Haveli and Daman and Diu',  'DN'],
  ['Delhi',  'DL'], ['Goa',  'GA'], ['Gujarat',  'GJ'], ['Haryana',  'HR'],[ 'Himachal Pradesh',  'HP'],
  ['Jammu and Kashmir',  'JK'],[ 'Jharkhand',  'JH'],[ 'Karnataka',  'KA'],[ 'Kerala',  'KL'],[ 'Lakshadweep', 'LD'],
 [ 'Madhya Pradesh',  'MP'],[ 'Maharashtra',  'MH'],['Manipur',  'MN'],[ 'Meghalaya','ML'],[ 'Mizoram',  'MZ'],
 ['Nagaland',  'NL'],[ 'Odisha',  'OR'],[ 'Puducherry',  'PY'],[ 'Punjab',  'PB'],['Rajasthan',  'RJ'],
 ['Sikkim',  'SK'],['Tamil Nadu', 'TN'],['Telangana',  'TG'],['Tripura','TR'],['Uttar Pradesh',  'UP'],
 ['Uttarakhand',  'UT'],['West Bengal',  'WB'],['Ladakh','LA']]

In [5]:
#FROM STATES LIST REPLACING STATE NAME WITH STATE ID

for i in range(len(states)):
    for j in range(len(change)):
        if states[i]==change[j][0]:
            states[i]=change[j][1]

In [6]:
#REPLACING THE STATE COLUMN OF DATA FRAME WITH STATE ID INPLACE STATE NAMES

df['State']=states

#STORING THE STATE ID AND DISTRICT NAME INTO DIFF LIST

stateid=list(df['State'])
distname=list(df['District'])

#CREATING A NEW LIST BY COMBINING STATE ID WITH DISTRICT NAME

distid=[]
for i in range(len(stateid)):
    distid.append(stateid[i]+'_'+distname[i])

In [7]:
#REPLACING THE DISTID COLUMN OF DATAFRAME WITH NEW DISTRICT IDS

df['distid']=distid

#REMOVING THE UNNECESSARY COLUMNS

del df['State']
del df['District']

In [8]:
#GROUPING THE DATFRAME BASED ON DISTRICT ID

gdf=df.groupby('distid')

In [9]:
#CREATING A NEW LIST THAT STORES THE DISTRICTID AND THE DATAFRAME THAT IS ASSOCIATED WITH THAT DISTRICT ID AS A ELEMENT

ml=[]
for i in gdf:
    ml.append([i[0],i[1]])

In [10]:
#FOR EACH  DISTRICTID DATAFRAME ADDING NEW ROWS WITH DATES FROM 2020-03-15 TO THE MIN DATE IN THAT DATAFRAME AND WITH CASES AS 0

for i in ml:
    startdate = np.datetime64('2020-03-15')
    while(startdate < min(i[1]['Date'])):
        i[1] =  i[1].append({'Date':startdate,'distid':i[0],'Confirmed':0,'Recovered':0,'Deceased':0},ignore_index=True)
        startdate = startdate + np.timedelta64(1,'D')
    i[1] = i[1].sort_values(by=['Date'], ascending=True)

In [11]:
#CREATING THE  WEEK FINAL DATAFRAME

weekfinaldf = pd.DataFrame(columns = ['districtid','weekid','cases'])

#FOR EACH DISTRICTID

for i in ml:
    week = 1
    
    #CREATING A TEMPORARY DATAFRAME
    
    tempdf = pd.DataFrame(columns = ['districtid','weekid','cases'])
    
    #STORING MIN AND MAX DATES FROM THE DISTRICT DATAFRAME
    
    startdate = np.datetime64(i[1][0:1]['Date'].to_string(index = False))
    enddate = np.datetime64(i[1][-1:]['Date'].to_string(index = False))
    
    #STORING THE DATE COLUMN OF THE DISTRICT DATAFRAME INTO A LIST
    
    datelist = list(i[1]['Date'])
    
    #STORING CONFIRMED,RECOVERED,DECEASED COLUMNS VALUES INTO A SEPERATE LISTS
    
    confirmedlist = list(i[1]['Confirmed'])
    recoveredlist=list(i[1]['Recovered'])
    deceasedlist=list(i[1]['Deceased'])
    
    #FROM START WEEK UNTIL THE LAST WEEK
    #CALCULATED THE NO OF ACTIVE CASES IN THAT WEEK BY FORMULA
    # NO OF ACTIVE CASES = CONFIRMED CASES-RECOVERED CASES-DECEASED CASES IN THE LAST DAY OF THAT WEEK
    #AND ADDING THE DISTRICT ID,WEEK ID AND ACTIVE CASES INTO THE TEMPORARY DATAFRAME
    
    while (startdate <= enddate-np.timedelta64(6,'D')):
        
        date2 = startdate + np.timedelta64(6,'D')
        ind_date2 = 0
        for j in range(0,len(datelist)):
            if datelist[j] == date2:
                ind_date2 = j
        tempdf = tempdf.append(pd.DataFrame({"districtid": i[0],"weekid":week,"cases":confirmedlist[ind_date2]-recoveredlist[ind_date2]-deceasedlist[ind_date2]}, index=[0]))
        
        
        week = week + 1
        
        date2 = startdate + np.timedelta64(10,'D')
        if date2<=enddate and ind_date2!=0:
            x=ind_date2+4
            tempdf = tempdf.append(pd.DataFrame({"districtid": i[0],"weekid":week,"cases":confirmedlist[x]-recoveredlist[x]-deceasedlist[x],}, index=[0]))
            
        elif date2<=enddate:
            
            tempdf = tempdf.append(pd.DataFrame({"districtid": i[0],"weekid":week,"cases":confirmedlist[ind_date2]-recoveredlist[ind_date2]-deceasedlist[ind_date2],}, index=[0]))
            
        if ind_date2!=0:
            while(ind_date2+7<=len(datelist)-1):
                week+=1
                ind_date2+=7
                tempdf = tempdf.append(pd.DataFrame({"districtid": i[0],"weekid":week,"cases":confirmedlist[ind_date2]-recoveredlist[ind_date2]-deceasedlist[ind_date2],}, index=[0]))
                if ind_date2+4<=len(datelist)-1:
                    week+=1
                    tempdf = tempdf.append(pd.DataFrame({"districtid": i[0],"weekid":week,"cases":confirmedlist[ind_date2+4]-recoveredlist[ind_date2+4]-deceasedlist[ind_date2+4],}, index=[0]))
                startdate=enddate
        week=week+1
        
        startdate = startdate + np.timedelta64(7,'D')
    
    #APPENDING THE TEMPORARY DATAFRAME TO THE WEEK FINAL DATAFRAME
    
    weekfinaldf = weekfinaldf.append(tempdf)

In [12]:
#STORED THE MONTH TO MONTH DATES IN A LIST

mydate=list(ml[0][1]['Date'][0:1])
next_month=[]
for i in range(18):
  if(mydate[0].month+i<=12):
    next_month.append(datetime(mydate[0].year, mydate[0].month+i, 14))
  else:
    next_month.append(datetime(mydate[0].year+1, mydate[0].month+i-12, 14))

In [ ]:
#CREATED  MONTH FINAL DATFRAME 

monfinaldf = pd.DataFrame(columns = ['districtid','monthid','cases'])

#FOR EACH DISTRICTID

for i in ml:
    month = 1
    
    #CREATING A TEMPORARY DATAFRAME
    
    tempdf = pd.DataFrame(columns = ['districtid','monthid','cases'])
    
     #STORING THE DATE COLUMN OF THE DISTRICT DATAFRAME INTO A LIST
    
    datelist = list(i[1]['Date'])
    
    #STORING CONFIRMED,RECOVERED,DECEASED COLUMNS VALUES INTO A SEPERATE LISTS
    
    confirmedlist = list(i[1]['Confirmed'])
    recoveredlist=list(i[1]['Recovered'])
    deceasedlist=list(i[1]['Deceased'])
    mno=0
    
    #FROM FIRST MONTH UNTIL THE LAST MONTH
    #CALCULATED THE NO OF ACTIVE CASES IN THAT MONTH BY FORMULA
    # NO OF ACTIVE CASES = CONFIRMED CASES-RECOVERED CASES-DECEASED CASES IN THE LAST DAY OF THAT MONTH
    #AND ADDING THE DISTRICT ID,MONTH ID AND ACTIVE CASES INTO THE TEMPORARY DATAFRAME
    
    while (mno<len(next_month)-1):
        date2 = next_month[mno+1]
        ind_date2 = 0
        for j in range(0,len(datelist)):
            if datelist[j] == date2:
                ind_date2 = j
        tempdf = tempdf.append(pd.DataFrame({"districtid": i[0],"monthid":month,"cases":confirmedlist[ind_date2]-recoveredlist[ind_date2]-deceasedlist[ind_date2]}, index=[0]))
        month = month + 1
        mno+=1
    
    #APPENDING THE TEMPORARY DATAFRAME TO THE WEEK FINAL DATAFRAME
    
    monfinaldf = monfinaldf.append(tempdf)

In [ ]:
#GROUPING THE WEEKFINAL DATAFRAME AND MONTH FINAL DATAFRAME SEPERATELY BASED ON DISTRICT ID

weekgdf=weekfinaldf.groupby('districtid')
mongdf=monfinaldf.groupby('districtid')

In [ ]:
#CREATING A NEW DATAFRAME TO STORE THE WAVE1-WEEKID AND WAVE2-WEEKID OF EACH DISTRICT

a=pd.DataFrame(columns=['districtid','wave1-weekid','wave2-weekid'])

#FOR EACH DISTRICTID 

for i,j in weekgdf:
    
    #SPLITTED THE DATAFRAME OF THAT DISTRICTID INTO TWO PARTS AND STORED THE MAX NO OF ACTIVE CASES FROM FIRST PART AND THE SECOND
    #PART INTO VARIABLES
    
    max1=max(j.iloc[0:int(len(j)/2),2])
    max2=max(j.iloc[int(len(j)/2):len(j),2])
    
    #FINDING THE WEEKID IN THE FIRST HALF THAT CORRESPONDING TO MAX ACTIVE CASES VALUE(MAX1)
    
    for k in range(int(len(j)/2)):
        if j.iloc[k,2]==max1:
            peak1week=j.iloc[k,1]
            
    #FINDING THE WEEKID IN THE SECOND HALF THAT CORRESPONDING TO MAX ACTIVE CASES VALUE(MAX2)
    
    for k in range(int(len(j)/2),len(j)):
        if j.iloc[k,2]==max2:
            peak2week=j.iloc[k,1]
            
    #APPENDING THESE WEEKIDS INTO  THE DATAFRAME CREATED ABOVE
    
    a=a.append(pd.DataFrame({'districtid':i,'wave1-weekid':peak1week,'wave2-weekid':peak2week},index=[0]))

In [ ]:
#CREATING A NEW DATAFRAME TO STORE THE WAVE1-MONTHID AND WAVE2-MONTHID OF EACH DISTRICT


b=pd.DataFrame(columns=['districtid','wave1-monthid','wave2-monthid'])

#FOR EACH DISTRICTID

for i,j in mongdf:
    
    #SPLITTED THE DATAFRAME OF THAT DISTRICTID INTO TWO PARTS AND STORED THE MAX NO OF ACTIVE CASES FROM FIRST PART AND THE SECOND
    #PART INTO VARIABLES
    
    max1=max(j.iloc[0:int(len(j)/2),2])
    max2=max(j.iloc[int(len(j)/2):len(j),2])
    
    #FINDING THE MONTHID IN THE FIRST HALF THAT CORRESPONDING TO MAX ACTIVE CASES VALUE
    
    for k in range(int(len(j)/2)):
        if j.iloc[k,2]==max1:
            peak1mon=j.iloc[k,1]
            
    #FINDING THE MONTHID IN THE SECOND HALF THAT CORRESPONDING TO MAX ACTIVE CASES VALUE
    
    for k in range(int(len(j)/2),len(j)):
        if j.iloc[k,2]==max2:
            peak2mon=j.iloc[k,1]
            
    #APPENDING THESE MONTHIDS INTO  THE DATAFRAME CREATED ABOVE
    
    b=b.append(pd.DataFrame({'districtid':i,'wave1-monthid':peak1mon,'wave2-monthid':peak2mon},index=[0]))

In [ ]:
#MERGED THE ABOVE CREATED DATAFRAMES BASED ON DISTRICT ID

merged = pd.merge(left=a, right=b, left_on='districtid', right_on='districtid')

In [ ]:
#CONVERTING THE ABOVE MERGED DATAFRAME INTO district-peaks.csv

merged.to_csv('district-peaks.csv',index=False)

In [ ]:
#GETTING ONLY DISTRICT IDS FROM WEEK FINAL DATAFRAME

distlist=list(weekfinaldf['districtid'])

#GETTING ONLY THE STATEIDS FROM THOSE DISTRICTIDS AND STORING IT IN A LIST

stateid=[]
for i in range(len(distlist)):
    stateid.append(distlist[i].split('_')[0])
    
#CREATING A NEW COLUMN IN THE WEEKFINAL DATAFRAME ONLY WITH STATEID

weekfinaldf['stateid']=stateid

#GETTING ONLY DISTRICT IDS FROM MONTH FINAL DATAFRAME

distlist=list(monfinaldf['districtid'])

#GETTING ONLY THE STATEIDS FROM THOSE DISTRICTIDS AND STORING IT IN A LIST

stateid=[]
for i in range(len(distlist)):
    stateid.append(distlist[i].split('_')[0])
    

#CREATING A NEW COLUMN IN THE MONTHFINAL DATAFRAME ONLY WITH STATEID

monfinaldf['stateid']=stateid

In [ ]:
#GROUPED THE WEEKFINAL DATAFRAME AND MONTHFINAL DATAFRAME BASED ON STATEID ,WEEKID AND STATEID,MONTHID RESPECTIVELY

stateweekgdf=weekfinaldf.groupby(['stateid','weekid'])
statemongdf=monfinaldf.groupby(['stateid','monthid'])

In [ ]:
#CREATING  A NEW DATAFRAME TO STORE STATEID,WEEKID,CASES OF EACH STATE

weekfinaldf = pd.DataFrame(columns = ['stateid','weekid','cases'])

#FOR EACH STATEID,WEEKID
#CALCULATING THE NO OF ACTIVE CASES IN THAT STATE IN THAT WEEK (BY SUMMING ALL CASES IN THE DATAFRAME CORRESPONDING TO THAT STATE AND WEEK)
#APPENDING STATEID,WEEKID,CASES IN THE ABOVE DATAFRAME

for i,j in stateweekgdf:
    weekfinaldf = weekfinaldf.append(pd.DataFrame({"stateid": i[0],"weekid":i[1],"cases":sum(j['cases'])}, index=[0]))

In [ ]:
#CREATING  A NEW DATAFRAME TO STORE STATEID,MONTHID,CASES OF EACH STATE

monfinaldf = pd.DataFrame(columns = ['stateid','monthid','cases'])

#FOR EACH STATEID ,MONTHID

#CALCULATING THE NO OF ACTIVE CASES IN THAT STATE IN THAT MONTH (BY SUMMING ALL CASES IN THE DATAFRAME CORRESPONDING TO THAT STATE AND MONTH)

#APPENDING STATEID,MONTHID,CASES IN THE ABOVE DATAFRAME

for i,j in statemongdf:
    monfinaldf = monfinaldf.append(pd.DataFrame({"stateid": i[0],"monthid":i[1],"cases":sum(j['cases'])}, index=[0]))

In [ ]:
#GROUPING THE ABOVE CREATED DATAFRAMES BASED ON STATEID

weekgdf=weekfinaldf.groupby('stateid')
mongdf=monfinaldf.groupby('stateid')

In [ ]:
#CREATED A NEW DATAFRAME TO STORE THE WAVE1-WEEKID AND WAVE2-WEEKID OF EACH STATE

a=pd.DataFrame(columns=['stateid','wave1-weekid','wave2-weekid'])

#FOR EACH STATEID 

for i,j in weekgdf:
    
    #SPLITTED THE DATAFRAME OF THAT STATEID INTO TWO PARTS AND STORED THE MAX NO OF ACTIVE CASES FROM FIRST PART AND THE SECOND
    #PART INTO VARIABLES
    
    max1=max(j.iloc[0:int(len(j)/2),2])
    max2=max(j.iloc[int(len(j)/2):len(j),2])
    
    #FINDING THE WEEKID IN THE FIRST HALF THAT CORRESPONDING TO MAX ACTIVE CASES VALUE(MAX1)
    
    for k in range(int(len(j)/2)):
        if j.iloc[k,2]==max1:
            peak1week=j.iloc[k,1]
            
    #FINDING THE WEEKID IN THE SECOND HALF THAT CORRESPONDING TO MAX ACTIVE CASES VALUE(MAX2)
    
    for k in range(int(len(j)/2),len(j)):
        if j.iloc[k,2]==max2:
            peak2week=j.iloc[k,1]
            
    #APPENDING THESE WEEKIDS INTO  THE DATAFRAME CREATED ABOVE
    
    a=a.append(pd.DataFrame({'stateid':i,'wave1-weekid':peak1week,'wave2-weekid':peak2week},index=[0]))

In [ ]:
#CREATED A NEW DATAFRAME TO STORE THE WAVE1-MONTHID AND WAVE2-MONTHID OF EACH STATE

b=pd.DataFrame(columns=['stateid','wave1-monthid','wave2-monthid'])

#FOR EACH STATEID

for i,j in mongdf:
    
    #SPLITTED THE DATAFRAME OF THAT STATEID INTO TWO PARTS AND STORED THE MAX NO OF ACTIVE CASES FROM FIRST PART AND THE SECOND
    #PART INTO VARIABLE
    
    max1=max(j.iloc[0:int(len(j)/2),2])
    max2=max(j.iloc[int(len(j)/2):len(j),2])
    
    #FINDING THE MONTHID IN THE FIRST HALF THAT CORRESPONDING TO MAX ACTIVE CASES VALUE(MAX1)
    
    for k in range(int(len(j)/2)):
        if j.iloc[k,2]==max1:
            peak1mon=j.iloc[k,1]
            
    #FINDING THE MONTHID IN THE SECOND HALF THAT CORRESPONDING TO MAX ACTIVE CASES VALUE(MAX2)
    
    for k in range(int(len(j)/2),len(j)):
        if j.iloc[k,2]==max2:
            peak2mon=j.iloc[k,1]
            
    #APPENDING THESE MONTHIDS INTO  THE DATAFRAME CREATED ABOVE
    
    b=b.append(pd.DataFrame({'stateid':i,'wave1-monthid':peak1mon,'wave2-monthid':peak2mon},index=[0]))

In [ ]:
#MERGED THE ABOVE CREATED DATAFRAMES BASED ON  STATEID

merged = pd.merge(left=a, right=b, left_on='stateid', right_on='stateid')

In [ ]:
#CONVERTING THE ABOVE MERGED DATAFRAME INTO state-peaks.csv

merged.to_csv('state-peaks.csv',index=False)

In [ ]:
#GROUPING THE WEEKFINAL DATAFRAME AND MONTHFINAL DATAFRAME BASED ON WEEKID AND MONTHID RESPECTIVELY

overallweekgdf=weekfinaldf.groupby(['weekid'])
overallmongdf=monfinaldf.groupby(['monthid'])

In [ ]:
#CREATING  A NEW DATAFRAME TO STORE OVERALLID,WEEKID,CASES OVERALL

weekfinaldf = pd.DataFrame(columns = ['overallid','weekid','cases'])

#FOR EACH WEEKID IN GROUPED WEEKFINAL DATAFRAME
#CALCULATING THE NO OF ACTIVE CASES OVERALL IN THAT WEEK (BY SUMMING ALL CASES IN THE DATAFRAME CORRESPONDING TO THAT  WEEK)
#APPENDED OVERALLID,WEEKID,CASES IN THE ABOVE DATAFRAME

for i,j in overallweekgdf:
    weekfinaldf = weekfinaldf.append(pd.DataFrame({"overallid": 1,"weekid":i,"cases":sum(j['cases'])}, index=[0]))

In [ ]:
##CREATING  A NEW DATAFRAME TO STORE OVERALLID,MONTHID,CASES OVERALL

monfinaldf = pd.DataFrame(columns = ['overallid','monthid','cases'])

#FOR EACH MONTHID IN GROUPED MONTHFINAL DATAFRAME
#CALCULATED THE NO OF ACTIVE CASES IN THAT THAT MONTH (BY SUMMING ALL CASES IN THE DATAFRAME CORRESPONDING TO THAT MONTH)
#APPENDED OVERALLID,MONTHID,CASES IN THE ABOVE DATAFRAME

for i,j in overallmongdf:
    monfinaldf = monfinaldf.append(pd.DataFrame({"overallid": 1,"monthid":i,"cases":sum(j['cases'])}, index=[0]))

In [ ]:
#GROUPED THE ABOVE CREATED DATAFRAMES BASED ON OVERALLID(LET US CALL THEM WEEKGDF AND MONGDF)

weekgdf=weekfinaldf.groupby('overallid')
mongdf=monfinaldf.groupby('overallid')

In [ ]:
#CREATED A NEW DATAFRAME TO STORE THE WAVE1-WEEKID AND WAVE2-WEEKID OVERALL

a=pd.DataFrame(columns=['overallid','wave1-weekid','wave2-weekid'])

#FOR EACH OVERALLID

for i,j in weekgdf:
    
    #SPLITTED THE DATAFRAME OF THAT OVERALLID INTO TWO PARTS AND STORED THE MAX NO OF ACTIVE CASES FROM FIRST PART AND THE SECOND
    #PART INTO VARIABLES
    
    max1=max(j.iloc[0:int(len(j)/2),2])
    max2=max(j.iloc[int(len(j)/2):len(j),2])
    
    #FINDING THE WEEKID IN THE FIRST HALF THAT CORRESPONDING TO MAX ACTIVE CASES VALUE(MAX1)
    
    for k in range(int(len(j)/2)):
        if j.iloc[k,2]==max1:
            peak1week=j.iloc[k,1]
            
    #FINDING THE WEEKID IN THE SECOND HALF THAT CORRESPONDING TO MAX ACTIVE CASES VALUE(MAX2)
    
    for k in range(int(len(j)/2),len(j)):
        if j.iloc[k,2]==max2:
            peak2week=j.iloc[k,1]
            
    #APPENDING THESE WEEKIDS INTO  THE DATAFRAME CREATED ABOVE
    
    a=a.append(pd.DataFrame({'overallid':i,'wave1-weekid':peak1week,'wave2-weekid':peak2week},index=[0]))

In [ ]:
#CREATED A NEW DATAFRAME TO STORE THE WAVE1-MONTHID AND WAVE2-MONTHID OVERALL

b=pd.DataFrame(columns=['overallid','wave1-monthid','wave2-monthid'])

#FOR EACH OVERALLID

for i,j in mongdf:
    
    #SPLITTED THE DATAFRAME OF THAT OVERALLID INTO TWO PARTS AND STORED THE MAX NO OF ACTIVE CASES FROM FIRST PART AND THE SECOND
    #PART INTO VARIABLES
    
    max1=max(j.iloc[0:int(len(j)/2),2])
    max2=max(j.iloc[int(len(j)/2):len(j),2])
    
    #FINDING THE MONTHID IN THE FIRST HALF THAT CORRESPONDING TO MAX ACTIVE CASES VALUE(MAX1)
    
    for k in range(int(len(j)/2)):
        if j.iloc[k,2]==max1:
            peak1mon=j.iloc[k,1]
    
    #FINDING THE MONTHID IN THE SECOND HALF THAT CORRESPONDING TO MAX ACTIVE CASES VALUE(MAX2)
    
    for k in range(int(len(j)/2),len(j)):
        if j.iloc[k,2]==max2:
            peak2mon=j.iloc[k,1]
            
    #APPENDING THESE MONTHIDS INTO  THE DATAFRAME CREATED ABOVE
    
    b=b.append(pd.DataFrame({'overallid':i,'wave1-monthid':peak1mon,'wave2-monthid':peak2mon},index=[0]))

In [ ]:
#MERGED THE ABOVE CREATED DATAFRAMES BASED ON  OVERALLID

merged = pd.merge(left=a, right=b, left_on='overallid', right_on='overallid')

In [ ]:
#CONVERTED THE ABOVE MERGED DATAFRAME INTO overall-peaks.csv

merged.to_csv('overall-peaks.csv',index=False)